In [ ]:
from IPython.display import display
import pandas as pd
import csv
import random
import re
# import pyperclip
import numpy as np
import seaborn as sns
import requests
import matplotlib, matplotlib.pyplot as plt
import pprint
import os
import dateutil
import datetime

pd.options.display.max_rows = 10
sns.set()
matplotlib.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'sans-serif']

data=pd.read_excel("赛题1数据材料\赛题1数据集\融资信息.xlsx")
data=data.replace('未披露',np.nan)
data.columns=['企业编号', '融资日期', '轮次', '$投资金额']
data

In [ ]:
def to_day(cell):
    if not isinstance(cell, str):
        return cell
    return (dateutil.parser.parse(cell) - datetime.datetime(2019, 3, 5)).days

df=data.set_index("企业编号")
df1 = df[['融资日期']].applymap(to_day).groupby('企业编号').mean().T
df1 = df1.reindex(columns=list(range(1001, 4001)))
df1

In [ ]:
data=data.set_index("轮次").loc[["IPO","战略投资","A轮","上市","主板定向增发",
                          "B轮","天使轮","天使轮","股权转让","被收购","C轮","新三板"],:].reset_index()
for i in range(len(data)):
    if ( type(data.iloc[i,3])==np.str and data.iloc[i,3][0]!="数"):
        if(data.iloc[i,3][-1]=="万"):
            data.iloc[i,3]=float(data.iloc[i,3][:-1])
        elif(data.iloc[i,3][-2]=="万"):
            data.iloc[i,3]=float(data.iloc[i,3][:-2])
        elif(data.iloc[i,3][-3]=="万"):
            data.iloc[i,3]=float(data.iloc[i,3][:-3])
        elif(data.iloc[i,3][-4]=="万"):
             data.iloc[i,3]=float(data.iloc[i,3][:-4])
    elif (type(data.iloc[i,3])==np.str and data.iloc[i,3][0]=="数"):
            data.iloc[i,3]=np.nan
data.columns=["轮次","企业编号","融资日期","$投资金额"]
data

In [ ]:
data=data.drop(columns="融资日期").set_index('企业编号')
result=data.groupby(['企业编号','轮次']).sum().unstack().T.reindex(columns=list(range(1001, 4001)))
result

In [ ]:
result.to_pickle('融资信息.pickle')